In [1]:
from datetime import datetime
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda:0


In [2]:
def seconds_to_time(seconds):
    s = int(seconds) % 60
    m = int(seconds) // 60
    if m < 1:
        return f'{s}s'
    h = m // 60
    m = m % 60
    if h < 1:
        return f'{m}m{s}s'
    return f'{h}h{m}m{s}s'

In [3]:
def train(model, loader, classes, epoch_count = 10, lr = 1e-3):
  loss_func = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = lr)
  model.train()

  start_time = datetime.now()

  for epoch in range(epoch_count):
    loss_acum = np.array([], dtype = np.float32)

    for data in loader:
      images = data[0].to(device)
      labels = torch.nn.functional.one_hot(data[1], classes).float().to(device)

      pred = model(images)
      loss = loss_func(pred, labels)
      loss_acum = np.append(loss_acum, loss.cpu().detach().numpy())

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    current_time = datetime.now()
    elapsed = seconds_to_time((current_time - start_time).total_seconds())
    print(f'Epoch: {epoch}, Time: {elapsed}, Loss: {np.mean(loss_acum)}')

In [4]:
def evaluate(model, loader):
  model.eval()

  correct_predictions = 0
  total_predictions = 0

  start_time = datetime.now()
  for data in loader:
    images = data[0].to(device)
    labels = data[1].to(device)

    with torch.no_grad():
      pred = model(images)
    label_pred = torch.argmax(pred, axis = 1)

    correct_predictions += torch.sum(labels == label_pred)
    total_predictions += images.shape[0]

  current_time = datetime.now()
  per_image = (current_time - start_time).total_seconds() / total_predictions
  print(f'Time: {per_image * 1000}ms, Accuracy: {correct_predictions / total_predictions}')

In [5]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.FashionMNIST('fmnist', train = True, download = True, transform = transforms_train)
test_dataset = torchvision.datasets.FashionMNIST('fmnist', train = False, download = True, transform = transforms_test)

num_workers = 2
batch_size = 128

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers = num_workers, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, num_workers = num_workers, shuffle = False)

print(f'Train: {len(train_dataset)}, Test: {len(test_dataset)}')

Train: 60000, Test: 10000


In [6]:
#@title Paprastas konvoliucinis tinklas
class SimpleConvNet(torch.nn.Module):
  def __init__(self, in_shape, out_classes):
    super().__init__()
    self.conv1 = torch.nn.Conv2d(in_shape[0], 4, (3, 3), padding = 'same')
    self.conv2 = torch.nn.Conv2d(4, 8, (3, 3), padding = 'same')
    self.fc1 = torch.nn.Linear(8 * (in_shape[1] // 4) * (in_shape[2] // 4), 128)
    self.fc2 = torch.nn.Linear(128, out_classes)

  def forward(self, x):
    y = torch.nn.ReLU()(self.conv1(x))
    y = torch.nn.MaxPool2d((2, 2), (2, 2))(y)
    y = torch.nn.ReLU()(self.conv2(y))
    y = torch.nn.MaxPool2d((2, 2), (2, 2))(y)
    y = torch.nn.Flatten()(y)
    y = torch.nn.ReLU()(self.fc1(y))
    y = self.fc2(y)
    return y

In [7]:
class_count = 5
# model = MlpNet(train_dataset[0][0].shape, 128, class_count).to(device)
model = SimpleConvNet(train_dataset[0][0].shape, class_count).to(device)
# model = ComplexConvNet(train_dataset[0][0].shape, class_count).to(device)
print(f'Parameter count: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}')

# train(model, train_loader, class_count, epoch_count = 30)
evaluate(model, train_loader)

Parameter count: 51,285
Time: 0.09490741666666667ms, Accuracy: 0.11505000293254852
